<h1 style="text-align:center">Héritage CAE</h1> 

Ce Notebook est un travail préparatif au développement d'un Simulateur de réforme de l'héritage basé sur les [travaux du CAE-ECO](https://www.cae-eco.fr/repenser-lheritage) 

In [69]:
%matplotlib widget
import numpy as np
import pandas as pd

# Import des données

## Héritage net par quantile

Sources : graphique [33 & 34 du Focus](doc/cae-focus077-2021.pdf)

In [70]:
# Read inheritance data
df_herit = pd.read_csv("data/heritage.csv", comment="#")
df_herit = df_herit.rename(columns={key: key.split("[")[0] for key in df.columns})
df_herit.quant = df_herit.quant
df_herit = df_herit.set_index("quant")
df_herit

,heritage
quant,
10.0,0
20.0,15000
30.0,47000
40.0,65000
50.0,65000
60.0,65000
70.0,84000
80.0,164000
90.0,288000


## Taux effectif d'imposition pour chaque scenario

Source: Graphique 28 du [Focus](doc/cae-focus077-2021.pdf), page 31. 

In [71]:
# Read taux effectif
df_taux = pd.read_csv("data/taux_effectifs.csv", comment="#")
df_taux = df_taux.rename(columns={key: key.split("[")[0] for key in df.columns})
df_taux.quant = df_taux.quant
df_taux = df_taux.set_index("quant")
df_taux

SCENARIOS = list(df_taux.columns)

print("Scenarios", SCENARIOS)

# Jointure des deux ensembles de données
df = df_herit.join(df_taux)
df

Scenarios ['actuel', 'assiette1-bareme1', 'assiette2-bareme2', 'assiette1-bareme3', 'assiette2-bareme4']


,heritage,actuel,assiette1-bareme1,assiette2-bareme2,assiette1-bareme3,assiette2-bareme4
quant,,,,,,
10.0,0,0.0,0.0,0.0,0.0,0.0
20.0,15000,0.0,0.0,0.0,0.0,0.0
30.0,47000,0.0,0.0,0.0,0.0,0.0
40.0,65000,0.0,0.0,0.0,0.0,0.0
50.0,65000,0.0,0.0,0.0,0.0,0.0
60.0,65000,0.0,0.0,0.0,0.0,0.0
70.0,84000,0.0,1.0,1.0,0.0,0.0
80.0,164000,1.0,1.0,2.0,0.0,0.0
90.0,288000,2.0,2.0,5.0,1.0,1.0


# Calculs 

## Volumes

On calcule les volumes moyen d'héritage pour chaque quantile.

In [73]:
# Compute volumes for each quantile
quantiles = np.insert(df.index.values, 0, [0])
df["volumes"]  = df.heritage.values * (quantiles[1:] - quantiles[0:-1]) / 100

display(df)

average = df.volumes.sum()
print("Héritage net moyen", df.volumes.sum())

base = 300*10**9 / average

print("Base calculée (sur hypothèse flux de succession 300 Mds€):", base)

,heritage,actuel,assiette1-bareme1,assiette2-bareme2,assiette1-bareme3,assiette2-bareme4,volumes
quant,,,,,,,
10.0,0,0.0,0.0,0.0,0.0,0.0,0.000
20.0,15000,0.0,0.0,0.0,0.0,0.0,1500.000
30.0,47000,0.0,0.0,0.0,0.0,0.0,4700.000
40.0,65000,0.0,0.0,0.0,0.0,0.0,6500.000
50.0,65000,0.0,0.0,0.0,0.0,0.0,6500.000
60.0,65000,0.0,0.0,0.0,0.0,0.0,6500.000
70.0,84000,0.0,1.0,1.0,0.0,0.0,8400.000
80.0,164000,1.0,1.0,2.0,0.0,0.0,16400.000
90.0,288000,2.0,2.0,5.0,1.0,1.0,28800.000


Héritage net moyen 190502.1119999995
Base calculée (sur hypothèse flux de succession 300 Mds€): 1574785.6905649465


### Quelle est la base de la distribution ?

<div style="background:yellow">
⚠ Question : Quelle est la "base" de ces quantiles ?

On trouve un héritage net moyen de 190k€.
Rapporté à un flux de 300 Md € [page 5 de la Note](./doc/cae-note069s.pdf), ça ferait une base de <b>1.5 millions de ??<> héritages / de décès / d'héritiers ? Ca semble loin des 720k naissances / 667k décès annuels.
</div>

## Recettes fiscales

Calcul des héritages bruts, avant imposition actuelle.
Puis calcul des recettes fiscales pour chaque scenario.

In [74]:
df["heritage_brut"] = df.volumes * 100 / (100 - df.actuel)

recettes = dict()
for scenar in SCENARIOS :
    total = (df.heritage_brut *  df[scenar] / 100).sum()
    recettes[scenar] = total
    
recettes = pd.DataFrame(
    recettes.items(), columns=["scenario", "recettes_moyennes"]).set_index("scenario")

# Différence de recette par rapport au scenario actuel
recettes["recettes_diff"] = recettes.recettes_moyennes - recettes.recettes_moyennes[0]
recettes

,recettes_moyennes,recettes_diff
scenario,,
actuel,8417.161523,0.000000
assiette1-bareme1,15396.690339,6979.528816
assiette2-bareme2,22521.564814,14104.403290
assiette1-bareme3,8451.240735,34.079211
assiette2-bareme4,17693.026402,9275.864879


### Quelle base ?

A nouveau se pose la question de la base.
Si on reprend la base précedente empirique 1.5 millions, on obtient :

In [75]:
# Recettes moyennes avec une  base 1.5 millions 
pd.merge(
    recettes.recettes_moyennes * base,
    recettes.recettes_diff * base, left_index=True, right_index=True)

,recettes_moyennes,recettes_diff
scenario,,
actuel,1.325523e+10,0.000000e+00
assiette1-bareme1,2.424649e+10,1.099126e+10
assiette2-bareme2,3.546664e+10,2.221141e+10
assiette1-bareme3,1.330889e+10,5.366745e+07
assiette2-bareme4,2.786272e+10,1.460750e+10


On retombe sur des recettes fiscales de **13 Mds €** :du même ordre de grandeur que **15 Mds €** (2022)
Par contre les recette supplémentaires sont un peu au delà de celles notées dans le [graphique 28 du focus](./doc/cae-focus077-2021.pdf) (page 31). Respectivement de 9, 19, 0 et 12 Mds€.